<a href="https://colab.research.google.com/github/jjkiljanski/biebrza-shrub-encroachment-analysis/blob/main/master_gee_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q earthengine-api geemap

import ee
import geemap

# Log in to your Google account for Earth Engine (you will see a link)
ee.Authenticate()

# Start Earth Engine
ee.Initialize(project='biebrza-encroachment-analysis')

In [32]:
import ipywidgets as widgets
from datetime import datetime

# ============================================================
# 1. Load Biebrzański National Park boundary
#    (from the WDPA – World Database on Protected Areas)
# ============================================================

wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")

# Filter areas whose NAME contains "Biebrza" (safe way to match spelling)
bpn = wdpa.filter(ee.Filter.stringContains("NAME", "Biebrza"))

print("Number of matching park polygons:", bpn.size().getInfo())

# Geometry for clipping satellite images
region_geom = bpn.geometry()


Number of matching park polygons: 3


In [33]:
def get_landsat_l2_sr():
    lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')  # do 2011
    le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')  # 1999–dzisiaj (SLC-off od 2003)
    lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')  # od 2013
    lc9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')  # od ~2021

    return lt5.merge(le7).merge(lc8).merge(lc9)

landsat = get_landsat_l2_sr()

In [34]:
# ================================================================
# Mask function for Landsat Collection 2 Level 2 Surface Reflectance
# ================================================================

def mask_l2_sr(image):
    """
    Applies the official USGS cloud/shadow/snow/saturation masks
    for Landsat Collection 2 Level-2 Surface Reflectance products.

    QA_PIXEL bits:
      Bit 0: Fill
      Bit 1: Dilated Cloud
      Bit 2: Cirrus
      Bit 3: Cloud
      Bit 4: Cloud Shadow

    QA_RADSAT:
      Indicates radiometric saturation in any band (values > 0 should be masked)
    """
    qa = image.select('QA_PIXEL')
    radsat = image.select('QA_RADSAT')

    # Bits 0–4 all must be zero (no fill, no clouds, no cirrus, no shadow)
    cloud_bits = int('11111', 2)
    qa_mask = qa.bitwiseAnd(cloud_bits).eq(0)

    # Radiometric saturation mask (value must be zero)
    sat_mask = radsat.eq(0)

    return image.updateMask(qa_mask).updateMask(sat_mask)

In [35]:
# ================================================================
# Correct band mapping for Landsat 5/7 (TM / ETM+)
# ================================================================

def prep_ls57(image):
    """
    Prepares Landsat 5 (TM) and Landsat 7 (ETM+) Level-2 SR images.

    Landsat 5/7 Surface Reflectance band names:
      SR_B1 = Blue
      SR_B2 = Green
      SR_B3 = Red
      SR_B4 = Near Infrared (NIR)
      SR_B5 = SWIR1
      SR_B7 = SWIR2
    """
    image = mask_l2_sr(image)

    # Rename bands to a unified spectral scheme (BLUE, GREEN, RED, NIR)
    return image.select(
        ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
        ['BLUE',  'GREEN', 'RED',  'NIR', 'SWIR1', 'SWIR2']
    )


# ================================================================
# Correct band mapping for Landsat 8/9 (OLI / OLI-2)
# ================================================================

def prep_ls89(image):
    """
    Prepares Landsat 8 and Landsat 9 Level-2 SR images.

    Landsat 8/9 Surface Reflectance band names:
      SR_B2 = Blue
      SR_B3 = Green
      SR_B4 = Red
      SR_B5 = Near Infrared (NIR)
      SR_B6 = SWIR1
      SR_B7 = SWIR2

    IMPORTANT:
      This mapping is DIFFERENT from Landsat 5/7!
      That’s why we cannot "guess" the mapping — we must explicitly map by sensor.
    """
    image = mask_l2_sr(image)

    return image.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],
        ['BLUE',  'GREEN', 'RED',  'NIR', 'SWIR1', 'SWIR2']
    )


# ================================================================
# 4. Build a single harmonized image collection combining L5/7/8/9
# ================================================================

# Landsat TM (5)  → use prep_ls57
lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').map(prep_ls57)

# Landsat ETM+ (7) → use prep_ls57
le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').map(prep_ls57)

# Landsat OLI (8)  → use prep_ls89
lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(prep_ls89)

# Landsat OLI-2 (9) → use prep_ls89
lc9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').map(prep_ls89)

# Unified L5+L7+L8+L9 collection with consistent band names
landsat = lt5.merge(le7).merge(lc8).merge(lc9)

In [42]:
#-------------------------------------------------
# annual composite with per-pixel obs_count
#   + per-year quality summary properties
# -------------------------------------------------

def annual_composite(year, geom, scale=30):
    """
    Build an annual median composite (May–Sep) and attach:
      - 'obs_count' band: number of valid observations per pixel
      - properties summarizing obs_count (mean, std, min, max)
      - property 'obs_hist': histogram of obs_count over geom
      - property 'image_count': number of images in the collection
    """
    year = int(year)
    start = ee.Date.fromYMD(year, 5, 1)
    end   = ee.Date.fromYMD(year, 9, 30)

    col = (landsat
           .filterBounds(geom)
           .filterDate(start, end))

    # Number of images in the collection for this year (anywhere in geom)
    image_count = col.size()

    # Per-pixel count of valid (unmasked) observations
    # Use any band (e.g. NIR) as they are all masked consistently.
    obs_count = col.select('NIR').count().rename('obs_count')

    # Median composite of reflectance bands
    median_img = col.median()

    # Combine reflectance + obs_count
    img = median_img.addBands(obs_count).clip(geom)

    # Global stats of obs_count over the geometry
    # (mean, stdDev, min, max) – good quick QC indicators.
    obs_stats = obs_count.reduceRegion(
        reducer=ee.Reducer.mean()\
                    .combine(ee.Reducer.stdDev(), sharedInputs=True)\
                    .combine(ee.Reducer.minMax(), sharedInputs=True),
        geometry=geom,
        scale=scale,
        maxPixels=1e12
    )

    # # Histogram of obs_count (how many pixels have 1, 2, 3... observations)
    # obs_hist = obs_count.reduceRegion(
    #     reducer=ee.Reducer.frequencyHistogram(),
    #     geometry=geom,
    #     scale=scale,
    #     maxPixels=1e12
    # )

    # Attach metadata as image properties
    img = img.set({
        'year': year,
        'image_count': image_count,
        'obs_mean':  obs_stats.get('obs_count_mean'),
        'obs_std':   obs_stats.get('obs_count_stdDev'),
        'obs_min':   obs_stats.get('obs_count_min'),
        'obs_max':   obs_stats.get('obs_count_max'),
    })

    return img

In [46]:
years = [1997,2015]#list(range(1997, 2026))
for y in years:
    img = annual_composite(y, region_geom)

    year        = img.get('year').getInfo()
    image_count = img.get('image_count').getInfo()
    obs_mean    = img.get('obs_mean').getInfo()
    obs_std     = img.get('obs_std').getInfo()
    obs_min     = img.get('obs_min').getInfo()
    obs_max     = img.get('obs_max').getInfo()

    print(f"\nYear {year}")
    print(f"  Images in collection       : {image_count}")
    print(f"  obs_count mean             : {obs_mean:.2f}")
    print(f"  obs_count std              : {obs_std:.2f}")
    print(f"  obs_count min / max        : {obs_min} / {obs_max}")


Year 1997
  Images in collection       : 35
  obs_count mean             : 8.87
  obs_count std              : 2.69
  obs_count min / max        : 2 / 18

Year 2015
  Images in collection       : 82
  obs_count mean             : 17.56
  obs_count std              : 4.85
  obs_count min / max        : 5 / 36


In [25]:
year = 1997
img = annual_images.filter(ee.Filter.eq('year', year)).first()

export_image = img.select(['RED','GREEN','BLUE','NIR','SWIR1','SWIR2']).toInt16()

task = ee.batch.Export.image.toDrive(
    image=export_image,
    description=f'biebrza_{year}_not_scaled',
    folder='GEE_Biebrza',
    fileNamePrefix=f'biebrza_{year}',
    scale=30,
    region=region_geom,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()